# Scaffold Generator

- Using a set of rules, generate a biochemical network allowing different species to associate with each other



In [1]:
from models.ModelBuilder import ModelBuilder
from models.Reaction import Reaction
from models.ReactionArchtype import ReactionArchtype
from models.ArchtypeCollections import *
from models.Utils import * 

import matplotlib.pyplot as plt
import roadrunner
import numpy as np
import pandas as pd

## Simple ABC Model 

Assuming Specie A recruits both B and C together, however, A can be bound to B, or C, or both B and C at the same time. However, B and C do not associate unless taken together by A. 

Thus, AB + C -> ABC and AC + B -> ABC

All Forward Reactions: 
- A + B -> AB 
- A + C -> AC 
- AB + C -> ABC 
- AC + B -> ABC 

Representing this: 
A: {B, C, [B, C]} # binds to B, C and can further associate B and C at the same time

If A binds either to B or C, then the representation would be: 
A: {B, C} # binds to either B or C but never together 

## Rule-based scaffold binding 

Makes an assumption that binding must occur through a specific binding site for the scaffold 

This means a scaffold will have a list of potential binding sites e.g. 
- Scaffold A: 
    - Surfaces: [B1, B2] # B1 and B2 are the names of the binding site
    - Prominence: [B3, ] # the shape of 'Scaffold A' makes it capable of binding to a B3 binding site 

Then, we define all species in the system in such manner 
- Specie B: 
    - Surfaces: [, ] # No binding site offered by this protein 
    - Prominence: [B1, B3] # binds to either proteins exposing a B1 or B3 binding site

In [2]:
class BindingSpecie: 
    def __init__(self, name: str, surfaces: list = None, prominences: list = None):
        '''
        utilise a binding mechanic to create complex ode models
            surfaces: list of strings, a surface is a binding site on the scaffold
            prominences: list of strings, a prominence is where 
        '''
        self.name = name
        self.surfaces = [] if surfaces is None else surfaces
        self.prominences = [] if prominences is None else prominences
        
    def set_surfaces(self, surfaces: list):
        self.surfaces = surfaces
        
    def set_prominences(self, prominences: list):
        self.prominences = prominences
        
    def add_surface(self, surface: str):
        self.surfaces.append(surface)
        
    def add_prominence(self, prominence):
        self.prominences.append(prominence)
        
    def get_surfaces(self):
        return self.surfaces
    
    def get_prominences(self):
        return self.prominences
    
    def is_legal_binding(self, other): # type: ignore
        '''
        check if two binding species can bind to each other
        '''
        for surface in self.surfaces:
            if surface in other.prominences:
                return True
        return False
    
    def __str__(self) -> str:
        return self.name
    
    
class BindingComplex:
    
    def __init__(self, specie_1, specie_2):
        '''
        create a binding complex from a list of binding species
        '''
        self.binding_species = []
        # sort in alphabetical order
        if isinstance(specie_1, BindingComplex):
            self.binding_species.extend(specie_1.binding_species)
        elif isinstance(specie_1, BindingSpecie):
            self.binding_species.append(specie_1)
        else: 
            raise ValueError("The first argument must be of type BindingSpecie or BindingComplex")
        
        if isinstance(specie_2, BindingComplex):
            self.binding_species.extend(specie_2.binding_species)
        elif isinstance(specie_2, BindingSpecie):
            self.binding_species.append(specie_2)
        else:
            raise ValueError("The second argument must be of type BindingSpecie or BindingComplex")

        self.binding_species = sorted(self.binding_species, key=lambda x: x.name)
        self.surfaces = []
        self.prominences = []
        
    def _set_surfaces_and_prominences(self):
        '''
        set all surfaces and prominences from the binding species, 
        however, remove the prominences / surfaces that are already bound between the species
        '''
        # check which binding surface is shared between the two species 
        
        
        
    def get_surfaces(self):
        pass 
    
    def get_prominences(self):
        pass
    
    def is_legal_binding(self, other):
        '''
        check if two binding complexes can bind to each other
        '''
        for specie in self.binding_species:
            if specie.is_legal_binding(other):
                return True
        return False
        
    def get_binding_species(self):
        return self.binding_species
    
    
    def __str__(self) -> str:
        # return the alphabetically ascending order of the binding species with '_' in between
        return "_".join([str(specie) for specie in self.binding_species])


def attempt_complex_formation(binding_specie_from, binding_specie_to):
    '''
    create a complex from two binding species
    '''
    if not binding_specie_from.is_legal_binding(binding_specie_to):
        raise ValueError("The two binding species cannot bind to each other")
    return BindingComplex([binding_specie_from, binding_specie_to])

def create_binding_reaction(binding_species: list):
    '''
    create a binding reaction between two binding species
    '''
    # ensure all items in the list are binding species
    for item in binding_species:
        if not isinstance(item, BindingSpecie):
            raise ValueError("All items in the list must be of type BindingSpecie")

    # first, based on the binding species, create a list of all possible binding reactions
    specie_binding_table = pd.DataFrame(-1, rows=binding_species, columns=binding_species)
    # check if there are any -1 values in the table
    while (specie_binding_table == -1).any().any():
        for i, row in enumerate(specie_binding_table):
            for j, value in enumerate(row):
                if value == -1:
                    formed = attempt_complex_formation(binding_species[i], binding_species[j])
                    if formed: 
                        # extend the row and column with the new complex
                        new_complex = BindingComplex([binding_species[i], binding_species[j]])
                        specie_binding_table.loc[new_complex] = -1
                        specie_binding_table[new_complex] = -1
                        specie_binding_table.loc[binding_species[i], binding_species[j]] = 1
                    else: 
                        specie_binding_table.loc[binding_species[i], binding_species[j]] = 0
                        
    # based on the table, create reactions between the binding species
    reactions = []
    for i, row in enumerate(specie_binding_table):
        for j, value in enumerate(row):
            if value == 1:
                reactions.append(Reaction(binding_species[i], binding_species[j], mass_action_21))
                
                
    

specie_1 = BindingSpecie("scaffold1", surfaces=["SH2", "SH3"], prominences=["P1"])
specie_2 = BindingSpecie('protein1', prominences=["SH2"])
specie_3 = BindingSpecie('protein2', prominences=["SH3"])
list_of_scaffolds = [specie_1, specie_2, specie_3]







